In [39]:
import pymupdf  # PyMuPDF
import fitz
import torch
from transformers import MarianTokenizer, AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoModelForSpeechSeq2Seq, AutoModelForQuestionAnswering
import chromadb
from chromadb.config import Settings

In [40]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with pymupdf.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [41]:
# Используем модель BART или другую T5 для генерации текста
tokenizer = AutoTokenizer.from_pretrained("bigscience/mt0-small")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-small")

# Инициализация базы данных ChromaDB
client = chromadb.Client(Settings(persist_directory="./chroma_db"))
collection = client.create_collection("pdf_documents")

UniqueConstraintError: Collection pdf_documents already exists

In [42]:
def embed_texts(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model.get_decoder()(**inputs).last_hidden_state.mean(dim=1)  # Среднее по всем токенам
    return embeddings

def index_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    embedding = embed_texts([text]).numpy().tolist()
    collection.add(documents=[text], embeddings=embedding)

In [ ]:
index_pdf('drive/MyDrive/233.pdf')

NameError: name 'tokenizer' is not defined

In [43]:
def retrieve_documents(query, k=5):
    query_embedding = embed_texts([query]).numpy().tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=k)
    return results['documents'][0]

In [44]:
import pandas as pd

In [45]:
def load_csv_data(file_path):
    """Загрузка данных из CSV-файла."""
    return pd.read_csv(file_path)

In [48]:
def rag_pipeline(query, csv_file_path, k=5):
    # Извлечение документов
    #retrieved_docs = retrieve_documents(query, k)
    # Чтение данных из CSV
    data = load_csv_data(csv_file_path)

    # Извлечение контекста (например, первые k строк)
    retrieved_docs = data.to_string()  # Получаем первые k контекстов

    context = "Искусственный интеллект - сфера науки о данных" # Объединяем документы в контекст

    # Создание входного текста для генерации
    input_text = f"Context: {context}\nQuery: {retrieved_docs}\nAnswer:"
    # Генерация ответа
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(input_ids)  # Генерация ответа

    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

In [47]:
query = "Что такое искусственный интеллект"
answer = rag_pipeline(query, 'test.csv')
print(answer)

Context: Искусственный интеллект - сфера науки о данных
Query:                                                                                                          user_input
0                                                   Как настроить норму внесения удобрений на распределителе ZG-TS?
1                                     Какие действия необходимо выполнить перед первым вводом в эксплуатацию ZG-TS?
2                                                                   Что означает система AutoTS и как её настроить?
3                                 Какие шаги нужно предпринять для проверки масляного фильтра распределителя ZG-TS?
4                                        Как настроить пограничное распределение с использованием системы BorderTS?
5                                   Какие меры безопасности необходимо соблюдать при работе с терминалом AmaTron 4?
6                                                                       Как подключить терминал AmaTron 4 к ISOBUS?
7        